# Modules, import mechanism, namespaces, scope(s)

In this chapter we will dive into one of the best features of python: the ``import`` mechanism. We will explain why modules are needed and how you can write one on your own.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modules,-import-mechanism,-namespaces,-scope(s)" data-toc-modified-id="Modules,-import-mechanism,-namespaces,-scope(s)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Modules, import mechanism, namespaces, scope(s)</a></span><ul class="toc-item"><li><span><a href="#Variables-scopes" data-toc-modified-id="Variables-scopes-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Variables scopes</a></span></li><li><span><a href="#Modules" data-toc-modified-id="Modules-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Modules</a></span></li><li><span><a href="#Namespaces" data-toc-modified-id="Namespaces-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Namespaces</a></span></li><li><span><a href="#sys.path" data-toc-modified-id="sys.path-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>sys.path</a></span></li><li><span><a href="#Reimporting-modules" data-toc-modified-id="Reimporting-modules-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Reimporting modules</a></span></li><li><span><a href="#Executing-modules-as-scripts" data-toc-modified-id="Executing-modules-as-scripts-7.6"><span class="toc-item-num">7.6&nbsp;&nbsp;</span>Executing modules as scripts</a></span></li><li><span><a href="#Take-home-points" data-toc-modified-id="Take-home-points-7.7"><span class="toc-item-num">7.7&nbsp;&nbsp;</span>Take home points</a></span></li><li><span><a href="#What's-next?" data-toc-modified-id="What's-next?-7.8"><span class="toc-item-num">7.8&nbsp;&nbsp;</span>What's next?</a></span></li></ul></li></ul></div>

## Variables scopes

The **scope** of a variable is the region of the program where the variable is valid, i.e. the "location" where the variable name can be used to refer to the data it links to. Let's consider this first example:

In [ ]:
def foo():
    i = 5
    print(i, 'in foo()')

print(i, 'global')  # will throw an error

It doesn't work because the name ``i`` is defined in the function which we defined but never called (you can see this because nothing is printed), so the function's statements were actually never run. Let's see if the following example works better:

In [ ]:
def foo():
    i = 5
    print(i, 'in foo()')

foo()

print(i, 'global')   # will throw an error

Here, the function is called and ``i`` is defined and equal to 5. However, the *scope* of the variable `i` is the block defined by the function ``foo``: outside of this scope the variable doesn't exist anymore. If this is understood, the following example might now be clearer?

In [ ]:
i = 1

def foo():
    i = 5
    print(i, 'in foo()')

print(i, 'global before foo()')

foo()

print(i, 'global after foo()')

The **global scope** refers to the highest scope level (the module or, in interactive mode, the interpreter). The function's scope in turn is called **local scope**. One says that the global scope prevails because what is possible in one direction isn't possible in the other:

In [ ]:
k = 2

def foo():
    print(k, 'is there a k in foo()?')

foo()

Yes, k is available in `foo()`: global variables are *also* available in the *nested* local scopes. They can be overridden locally (like in the example above), but this change won't be visible at the global scope level.

<img src="../img/logo_ex.png" align="left" style="width:1em; height:1em;"> **Exercise**: do you think there is a way to really overwrite the value of a global variable from a local scope? If yes, can you find it?

## Modules 

Python **modules** are used to store a collection of functions and variables that can be imported into other modules or into the main module (the collection of variables that you have access to in a script executed at the top level or in calculator mode).

A module is a file containing Python definitions and statements. The file name is the module name with the suffix .py appended. Within a module, the module’s name (as a string) is available as the value of the global variable `__name__`. For instance, use your favorite text editor to create a file called `greetings.py` in the current directory with the following contents:

```python
"""A module to say hi in several langages."""

base = {
    'en':'Hi {}!', 
    'fr':'Salut {}!', 
    'cn':'你好 {}!'
}

def say_hi(name, lang='en'):
    """Say hi in the language of your choice."""
    if lang not in base:
        print('Language unknown: {}'.format(lang))
    print(base[lang].format(name))
    
def add_lang(key, phrase):
    """Add a language to the list."""
    global base
    if key in base:
        print('Language {} already available'.format(key))
        return
    base[key] = phrase
```

Now enter the Python interpreter and import this module with the following command:

In [ ]:
import greetings

By importing the module, we have access to the functions it contains:

In [ ]:
greetings.say_hi('Fabi')
greetings.say_hi('Xiao Li', lang='cn')

But also to the global variables defined at the top level:

In [ ]:
greetings.base

Now use the new `global` statement we just discovered:

In [ ]:
greetings.add_lang('de', 'Hallo {}!')

In [ ]:
greetings.say_hi('Boris', lang='de')

## Namespaces 

Global variables in modules or scripts are useful, but they should be used with care. Indeed, since they are available in everywhere in the module, the **namespace** can quickly become "polluted": for large modules, it becomes hard to keep track of which variable is available where. This can become even more complicated when new modules with their own new variables and functions are used. 

Fortunately, the global scope is constrained to the current script or module, and using external modules is unlikely to cause confusion thanks to the ``import`` mechanism:

In [ ]:
pi = 3.14
import math
print(math)
print(pi, math.pi)

By importing the ``math`` module (available in python's **standard library**), we have access to new variables and functions. What are they? We can use the built-in `dir()` function to find out:

In [ ]:
print(dir(math))

<img src="../img/logo_ex.png" align="left" style="width:1em; height:1em;"> **Exercise**: now ask the documentation what ``dir`` does (remember how? Simply type ``? dir`` in the ipython interpreter, or use a search engine).

Like our `greetings` module, the `math` module contains functions (like ``math.radians``) as well as variables (``math.nan``). Some of the variables have a prefix of two underscores ``__``. We'll get back to them soon.

There are several ways to import (parts of) a module. See the four examples:

In [ ]:
# A
# Import an entire module and access its attributes with a "dot"
import math
math.sin(math.pi/2)

In [ ]:
# B
# Same as A, but store the module under a new variable named "ma"
import math as ma
ma.sin(ma.pi/2)

In [ ]:
# C
# Import the attributes sin and pi *only*
from math import sin, pi
sin(pi/2)

In [ ]:
# D
# Import everything from the module and add it in our global scope
from math import *
sin(pi/2)

All four options lead to the same result: they compute the sinus of π/2.

Now, which one to use? It is up to you, but there are some implicit rules (conventions) which are widely accepted:
  1. In case of doubt, use option A. It is the most readable and the most *explicit*.
  2. The exception to the first rule is when the library has a naming convention for its acronym. A good example is `numpy`, which [recommends to use](https://docs.scipy.org/doc/scipy/reference/tutorial/general.html) ``import numpy as np``.
  3. Option C might be useful if the names you are importing are very explicit, and if you expect to use them often in your script. Otherwise, it is not recommended.
  4. Option D is bad. Don't use it. 
  5. You can make an exception to rule 4 when working in the command line and exploring data. It should not be used in modules or scripts, though. 

<img src="../img/logo_ex.png" align="left" style="width:1em; height:1em;"> **Exercise**: try to find arguments as to why is option D a bad idea. **If you can't find any reason, ask me next week**!

## sys.path 
<a id='path'></a>

But how does python know where to look for modules when you type ``import mymodule``? Well, it relies on a mechanism very similar to linux's ``PATH`` environment variable. Remember this one? Within python, you can ask in which directories the interpreter will look for modules with [sys.path](https://docs.python.org/3/library/sys.html#sys.path):

```python
import sys
sys.path

['',
 '/scratch/c707/c7071047/miniconda3/bin',
 '/scratch/c707/c7071047/miniconda3/lib/python36.zip',
 '/scratch/c707/c7071047/miniconda3/lib/python3.6',
 '/scratch/c707/c7071047/miniconda3/lib/python3.6/lib-dynload',
 '/scratch/c707/c7071047/miniconda3/lib/python3.6/site-packages',
 '/scratch/c707/c7071047/miniconda3/lib/python3.6/site-packages/cycler-0.10.0-py3.6.egg',
 '/scratch/c707/c7071047/miniconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/c7071047/.ipython']
```

Similar to the linux ``PATH`` mechanism, python will look into each of these directories *in order*. The first directory is always the current working directory, the rest of the list may vary depending on your environment. When a file called ``mymodule.py`` is found, it is imported once (and only once) and added to the ``sys.modules`` variable, effectively telling python that the module has already been imported. **This means that if you change the file and import it again, nothing will change**. Fortunately, there are ways to avoid this behavior (see below).

As you can see, there are many folders related to ``miniconda``, the tool we used to install python. This makes sense, because we want python to look for modules related to *our* installation of python. In particular, the ``site-packages`` folder is of interest to us. If you look into this folder (remember how?) you'll find the many packages we already installed together last week.

You can edit ``sys.path`` at your wish and add new folders to it, exactly as you would with the ``PATH`` environment variable in linux. In practice, however, it is recommended to use standard folders to install your packages instead of messing around with ``sys.path``.

<img src="../img/logo_ex.png" align="left" style="width:1em; height:1em;"> **Exercise**: let's trick with ``sys.path`` a little: create a fake module called ``matplotlib.py`` in your current working directory. Open a python interpreter and ``import matplotlib``. Which of the modules (the official or the fake one) is loaded? 

Because of this "feature", it is important to find meaningful (and unique) names for your own modules (don't forget to delete the fake ``matplotlib`` module!).

*Note: the "naming trick" above does not work with modules of the standard library, because built-in modules are imported as part of the interpreter's startup process.*

## Reimporting modules 

Of course, there is a lot more behind an `import` statement than the short summary I made above (see the [documentation](https://docs.python.org/3/reference/simple_stmts.html#import) for a more comprehensive overview). What you should remember however is that an import statement is not computationally cheap: it has to look for a file, (maybe) compile it to bytecode, add it to a registry... Actually, for all what it does, it is very fast already! But since `import` statements are used all the time in Python, it is a good idea to make them cheap.

However, in interactive development, it might be useful to make changes to a module and having them available at the command line without having to restart a fresh Python interpreter. There are two mechanisms available to you:

1. Using Python's [importlib.reload()](https://docs.python.org/3/library/importlib.html#importlib.reload) function:

In [ ]:
import importlib
importlib.reload(greetings)

This can reimport a module which has been imported before (i.e. it won't do the search for it again). But this has to be done *each time* you change a module, making it quite cumbersome in interactive mode. Here again, ipython comes with a more flexible solution:

2. Using ipython's [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html) extension:

In [ ]:
%load_ext autoreload
%autoreload 2

From now on, `autoreload` will make sure that all modules are reloaded automatically before entering the execution of code typed at the IPython prompt.

## Executing modules as scripts 

You've already learned that importing modules actually *runs* them, the same way as `$ python mymodule.py` (from the linux terminal) or `%run mymodule.py` (in ipython) would. This can have some undesired consequences: what if, for example, you would like certain things to happen when the module is run as script but *not* when being imported?

There is a mechanism that allows to do exactly this. Add the following block of code at the end of the `greetings.py` module:

```python
if __name__ == '__main__':
    # execute only if run as a script
    import sys
    nargs = len(sys.argv)
    if nargs == 3:
        say_hi(sys.argv[1], lang=sys.argv[2])
    else:
        print('Syntax:')
        print('%run greetings.py name language')
        print('Languages available: {}'.format(list(base)))
    
```

Now you can execute your script like this:

In [ ]:
%run greetings.py Fabi fr

But importing it will produce no output:

In [ ]:
import greetings

How does this work exactly? The important bit is the `if __name__ == '__main__':` line. `__name__` (and all other attributes starting and ending with two underscores) is "reserved" by the language. They are no different than other variables, but you should not erase or replace them (for now), since they contain useful information and might be used by other tools. For example:

In [ ]:
greetings.__doc__  # the documentation of the module as string

In [ ]:
greetings.__name__  # the name of the module as string

Now, back to our script: the statement `if __name__ == '__main__'` is going to be false when the module is imported (since the name is `greetings`), but obviously true when the module is run as script. Indeed, `__name__` is set to `__main__` when executed.

## Take home points

- variables are defined in a **scope**, and we learned about two scopes: **local** and **global**. There are precise rules as to how each scope can interact with the other.
- the import mechanism is useful to avoid **namespace pollution**. Modules come with their own namespace, and it is recommended to keep each module's namespace clean (i.e. no ``from XYZ import *``)
- python borrowed the ``PATH`` mechanism from linux and uses a similar logic to look for modules and packages. Installing a new module is therefore super easy: just put a ``mymodule.py`` at the right place. (and don't mess with the existing modules!)
- the `%autoreload` command from ipython allows to dynamically change a module while using it at the command line
- when writing scripts, always put the code that needs to be executed in an `if __name__ == '__main__'` block: that way, your script can still be imported for the functions it proposes

## What's next? 

Back to the [table of contents](00-Introduction.ipynb#ctoc), or [jump to the next chapter](08-Tips-and-Tricks.ipynb).